In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import Normalizer

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier

from keras.layers import Dense
from keras.models import Sequential

import random


from sklearn.metrics import roc_curve, auc

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('/content/train.csv')
# for oputma
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features와 Target 분리
X = df[['cluster0', 'cluster1', 'cluster2', 'cluster3', 'cluster4', 'cluster5']]
y = df['playoffs']


# 데이터셋 분리 (train/test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train 데이터셋을 train/val로 다시 분리
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Note: 0.25 * 0.8 = 0.2, so 20% of the original dataset is now used as validation

# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [2]:
from sklearn.metrics import precision_score
import tensorflow as tf

# 데이터셋 분리 (train/test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# 모델 탐색 파라미터
activations = ['relu', 'tanh']
learning_rates = [0.0001, 0.001, 0.01]
kernel_initializers = ['he_uniform', 'glorot_uniform']
dropout_rates = [0.3, 0.5]
batch_sizes = [1, 5, 9, 18, 27]
layers_configurations = [(128, 64, 32), (64, 32)]

best_precision = 0
best_model = None
best_params = {}

# 모델 탐색
for layers in layers_configurations:
    for activation in activations:
        for learning_rate in learning_rates:
            for kernel_initializer in kernel_initializers:
                for dropout_rate in dropout_rates:
                    for batch_size in batch_sizes:
                        print(f"Training with layers={layers}, activation={activation}, learning_rate={learning_rate}, kernel_initializer={kernel_initializer}, dropout_rate={dropout_rate}, batch_size={batch_size}")

                        # 최적의 모델 생성
                        random_state = 113
                        np.random.seed(random_state)
                        random.seed(random_state)
                        tf.random.set_seed(random_state)
                        model = create_custom_mlp_model(
                            layers=layers,
                            activation=activation,
                            use_leakyrelu=True,
                            use_batchnorm=True,
                            dropout_rate=dropout_rate,
                            optimizer='adam',
                            learning_rate=learning_rate,
                            kernel_initializer=kernel_initializer
                        )

                        # 모델 학습
                        history = model.fit(
                            X_train, y_train,
                            batch_size=batch_size,
                            epochs=50,
                            validation_data=(X_val, y_val),
                            callbacks=[
                                tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
                                tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
                            ],
                            verbose=0
                        )

                        # 모델 평가
                        y_pred = (model.predict(X_test) > 0.5).astype("int32")
                        final_precision = precision_score(y_test, y_pred)
                        print(f"테스트 셋 precision: {final_precision:.4f}")

                        # 최적 모델 저장
                        if final_precision > best_precision:
                            best_precision = final_precision
                            best_model = model
                            best_params = {
                                'layers': layers,
                                'activation': activation,
                                'learning_rate': learning_rate,
                                'kernel_initializer': kernel_initializer,
                                'dropout_rate': dropout_rate,
                                'batch_size': batch_size
                            }

print("최적 모델의 정밀도:", best_precision)
print("최적 하이퍼파라미터:", best_params)



In [9]:
import numpy as np
from sklearn.metrics import precision_score
import tensorflow as tf

# create_custom_mlp_model 함수 정의
def create_custom_mlp_model(layers, activation, use_leakyrelu, use_batchnorm, dropout_rate, optimizer, learning_rate, kernel_initializer):
    model = tf.keras.Sequential()
    for units in layers:
        model.add(tf.keras.layers.Dense(units, kernel_initializer=kernel_initializer))
        if use_batchnorm:
            model.add(tf.keras.layers.BatchNormalization())
        if use_leakyrelu:
            model.add(tf.keras.layers.LeakyReLU())
        else:
            model.add(tf.keras.layers.Activation(activation))
        model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# 초기 변수 설정
best_precision = 0
best_params = {}
precisions = []
random_state = 113

# 10번 반복 학습 및 평가
for i in range(10):
    # 최적의 하이퍼파라미터 설정
    layers = (128, 64, 32)
    activation = 'relu'
    use_leakyrelu = True
    use_batchnorm = True
    dropout_rate = 0.3
    optimizer = 'adam'
    learning_rate = 0.0001
    kernel_initializer = 'he_uniform'
    batch_size = 5
    epochs = 50
    np.random.seed(random_state)
    random.seed(random_state)
    tf.random.set_seed(random_state)

    # 모델 생성
    model = create_custom_mlp_model(
        layers=layers,
        activation=activation,
        use_leakyrelu=use_leakyrelu,
        use_batchnorm=use_batchnorm,
        dropout_rate=dropout_rate,
        optimizer=optimizer,
        learning_rate=learning_rate,
        kernel_initializer=kernel_initializer
    )

    # 모델 학습
    model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
        ],
        verbose=0
    )

    # 모델 평가
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    precision = precision_score(y_test, y_pred)
    precisions.append(precision)
    print(f"Run {i + 1}: Test Precision = {precision:.4f}")

    # 최적 정밀도와 하이퍼파라미터 저장
    if precision > best_precision:
        best_precision = precision
        best_params = {
            'layers': layers,
            'activation': activation,
            'learning_rate': learning_rate,
            'kernel_initializer': kernel_initializer,
            'dropout_rate': dropout_rate,
            'batch_size': batch_size,
            'epochs': epochs
        }

# 전체 정밀도 결과 출력
print("\nAll precisions:", precisions)
print("Average Precision:", np.mean(precisions))
print("최적 정밀도:", best_precision)
print("최적 하이퍼파라미터:", best_params)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Run 1: Test Precision = 0.7931
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Run 2: Test Precision = 0.7931
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Run 3: Test Precision = 0.7931
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Run 4: Test Precision = 0.7931
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Run 5: Test Precision = 0.7931
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Run 6: Test Precision = 0.7931
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Run 7: Test Precision = 0.7931
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Run 8: Test Precision = 0.7931
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Run 9: Test Precision = 0.7931
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Run 10: Test Precision = 0.7931

All precisions: [0.7931034482758621, 0.7931034482758621, 0.7931034482758621, 0.7931034482758621, 0.7931034482758621, 0.7931034482758621, 0.7931034482758621, 0.7931034482758621, 0.7931034482758621, 0.7931034482758621]
Average Precision: 0.7931034482758621
최적 정밀도: 0.7931034482758621
최적 하이퍼파라미터: {'layers': (12

In [11]:
# 모델 저장
model.save("final_model.h5")
print("모델이 'final_model.h5'로 저장되었습니다.")
model.summary()

from google.colab import files
files.download("final_model.h5")



모델이 'final_model.h5'로 저장되었습니다.


Model: "sequential_51"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_204 (Dense)                    │ (None, 128)                 │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_153              │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_48 (LeakyReLU)           │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_153 (Dropout)                │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_205 (Dense)                    │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_154              │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_49 (LeakyReLU)           │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_154 (Dropout)                │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_206 (Dense)                    │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_155              │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_50 (LeakyReLU)           │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_155 (Dropout)                │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_207 (Dense)                    │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,589 (139.02 KB)

 Trainable params: 11,713 (45.75 KB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 23,428 (91.52 KB)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>